In [1]:
import sys
from os.path import abspath, dirname
sys.path.append(abspath(dirname('qcc.ipynb')))
from module import mod2
mod2 = mod2()

In [2]:
import numpy as np
from numpy.linalg import matrix_power

def s_matrix(dim):
    matrix = np.eye(dim)
    matrix = np.concatenate((matrix[:, -1:], matrix[:, :-1]), axis=1)
    return matrix

def x_matrix(l, m):
    return np.kron(s_matrix(l), np.eye(m))

def y_matrix(l, m):
    return np.kron(np.eye(l), s_matrix(m))

class QuasiCyclicCode:
    def __init__(self, l, m, polynomial_a, polynomial_b) -> None:
        assert len(polynomial_a) == len(polynomial_b) == 3
        self.a_matrices = [
            matrix_power(m_function(l, m), poly) % 2
            for m_function, poly in zip([x_matrix, y_matrix, y_matrix], polynomial_a)
        ]
        self.a_matrix = sum(self.a_matrices) % 2
        self.b_matrices = [
            matrix_power(m_function(l, m), poly) % 2
            for m_function, poly in zip([y_matrix, x_matrix, x_matrix], polynomial_b)
        ]
        self.b_matrix = sum(self.b_matrices) % 2
        self.hx = np.concatenate([self.a_matrix, self.b_matrix], axis=1)
        self.hz = np.concatenate([self.b_matrix.T, self.a_matrix.T], axis=1)
        pass

qcc = QuasiCyclicCode(6, 6, [3, 1, 2], [3, 1, 2])
print(qcc.hx.shape, qcc.hz.shape)
print(((qcc.hx @ qcc.hz.T) % 2).nonzero())
hax = np.concatenate([sum(qcc.a_matrices[1:]) % 2, qcc.b_matrices[2]], axis=1)
hbx = np.concatenate([qcc.a_matrices[0], sum(qcc.b_matrices[:2]) % 2], axis=1)
haz = np.concatenate([qcc.b_matrices[2].T, sum(qcc.a_matrices[1:]).T % 2], axis=1)
hbz = np.concatenate([sum(qcc.b_matrices[:2]).T % 2, qcc.a_matrices[0].T], axis=1)
# assert np.allclose((hax + hbx) % 2, qcc.hx)
# print(hax.sum(1), haz.sum(1), (hax + haz).sum(0))
# print(hbx.sum(1), hbz.sum(1), (hbx + hbz).sum(0))

(36, 72) (36, 72)
(array([], dtype=int64), array([], dtype=int64))


In [7]:
import torch
hx, hz = torch.from_numpy(qcc.hx), torch.from_numpy(qcc.hz)
print(mod2.rank(hx))
print(mod2.rank(hz))

a = mod2.indep(hx)
print(a.shape)
b = mod2.indep(hz)
print(b.shape)
# c = torch.zeros_like(a)
# PCM = torch.vstack(torch.hstack([a, c]), torch.hstack([c, b]))
# print(PCM.size())


30
30
torch.Size([30, 72])
torch.Size([30, 72])
